In [1]:
!pip install tensorflow_text

In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import os
import pandas as pd
import numpy as np

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [4]:
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = "/content/drive/MyDrive/project_data"
OUTPUT_DIR = "/content/drive/MyDrive/project_data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df_train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"), header=0)
df_train = df_train.dropna(how="any")
df_train["targetcat_encoded"] = df_train["targetcat"].astype('category').cat.codes
df_train = df_train.rename(columns={"targetcat_encoded": "label", "description": "text"})

In [6]:
col = 'targetcat'
class MyDict(dict):
    def __missing__(self, key):
        return None
mapping_dict = MyDict({k: v for k, v in zip(df_train['targetcat'], df_train['label'])})
mapping_dict

{'Меню-холдеры': 1109,
 'Слуховые аппараты': 1879,
 'Соусы': 1902,
 'Папки': 1399,
 'Свечи зажигания': 1776,
 'Комплектующие для окон и дверей': 862,
 'Бокалы и фужеры': 286,
 'Ингаляторы': 729,
 'Семечки': 1815,
 'Портативная электроника': 1548,
 'Наборы смесителей': 1231,
 'Велошлемы': 342,
 'Одежда для сферы обслуживания': 1329,
 'Мобильные телефоны': 1136,
 'Бумажная продукция': 303,
 'Все для ламинирования ресниц': 398,
 'Архивные короба и боксы': 227,
 'Простыни': 1618,
 'Барьерная защита для тату оборудования': 245,
 'Уплотнитель для окон и дверей': 2202,
 'Электронные сигареты и аксессуары': 2432,
 'Электролобзики': 2424,
 'Зубная щетка': 697,
 'Компьютерные столы': 892,
 'Детские горшки, сиденья и подставки': 528,
 'Дерево и пиломатериалы': 511,
 'Затирочные смеси (Фуга)': 677,
 'Адаптеры и контроллеры': 83,
 'Мороженое': 1159,
 'Картриджи для тату машинок': 792,
 'Оптические диски': 1349,
 'Ноутбуки': 1298,
 'Парфюмерия': 1418,
 'Автокраски': 45,
 'Гейнеры': 432,
 'Средства д

In [7]:
df_valid = pd.read_csv(os.path.join(INPUT_DIR, "valid.csv"), header=0)
df_valid = df_valid.dropna(how="any")
df_valid["targetcat_encoded"] = df_valid["targetcat"].map(mapping_dict)
df_valid = df_valid.rename(columns={"targetcat_encoded": "label", "description": "text"})
df_valid = df_valid.dropna(how="any")
df_valid["label"] = df_valid["label"].astype(int)

In [8]:
df_train.head()

,text,title,targetcat,label
0,Держатель для карточки меню\n\nМатериал: нержа...,Держатель для счетов Empire 9712Е,Меню-холдеры,1109
1,Аккумуляторный Слуховой Аппарат в виде блютуз ...,Аккумуляторный Слуховой Аппарат Axon A-155 Уси...,Слуховые аппараты,1879
2,Tabasco — самый известный и узнаваемый из остр...,Соус из красного перца Tabasco Pepper Sauce 60...,Соусы,1902
3,Призначена для зручного носіння та зберігання ...,"Папка для зош. B5 ""Yes"" №491728 ""LOL LOL Sweet...",Папки,1399
4,Центральный электрод: платина;\nБоковой электр...,Cвеча зажигания Denso PT16VR13 (5068),Свечи зажигания,1776


In [9]:
df_valid[df_valid.label == 1902]

,text,title,targetcat,label
1988,"Cоус маринад Терияки Miyata - сладко-соленый, ...",Соус маринад Терияки Miyata 150 мл,Соусы,1902
4967,Сальса острая - традиционный соус мексиканско...,Сальса Hot Salsa Dip Antica Cantina 300 г,Соусы,1902
7652,"Соус, который облегчает приготовление традицио...",Соус Кімчі SaP 1 л,Соусы,1902
8089,Кетчуп томатний «Класик » пастеризований ТМ «В...,Упаковка Кетчуп томатний ВІКІ Класик пастеризо...,Соусы,1902
10618,Кетчуп Торчин Нежный- самый популярный кетчуп ...,Упаковка кетчупа ТОРЧИН Нежный 540 г х 16 шт (...,Соусы,1902
14920,Соевый соус Master Souse может добавляться как...,Набор соевых соусов Master Souse №8 Классическ...,Соусы,1902
18190,Миллионы людей во всём мире знают и используют...,Соус соевый Kikkoman 1 л (8715035110809),Соусы,1902
19251,Соусы J'ELITE Новый кисло-сладкий соус к мясу ...,Соус к мясу J'ELITE Абрикосовый карри с имбирём,Соусы,1902
20375,Смесь цельных горчичных зерен с богатым букето...,Упаковка горчицы ТОРЧИН Дижонская 130 г х 24 ш...,Соусы,1902
21056,Бренд Spilva известен в Латвии с 1992 года и у...,Упаковка coycа Spilva Asia Chilli 300 г х 2 шт...,Соусы,1902


In [10]:
hub_layer = hub.KerasLayer(embed, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [15]:
category_counts = len(df_train['label'].value_counts())
category_counts

2462

In [16]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'
embedding_layer = hub.KerasLayer(module_url)
hidden_layer = tf.keras.layers.Dense(256, activation='relu')
output_layer = tf.keras.layers.Dense(category_counts, activation='softmax')

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = embedding_layer(tf.squeeze(tf.cast(inputs, tf.string)))
x = hidden_layer(x)
outputs = output_layer(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.cast_1 (TFOpLambda)      (None, 1)                 0         
                                                                 
 tf.compat.v1.squeeze_1 (TFO  None                     0         
 pLambda)                                                        
                                                                 
 keras_layer_2 (KerasLayer)  (None, 512)               68927232  
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 2462)              632734    
                                                           

In [ ]:
history = model.fit(x=df_train['text'],
                    y=df_train['label'],
                    validation_data = (df_valid['text'], df_valid['label']),
                    epochs=10,
                    verbose=1)

Epoch 1/10
3335/3878 [========================>.....] - ETA: 20s - loss: 3.7376 - sparse_categorical_accuracy: 0.3421

In [ ]:
model.save(os.path.join(OUTPUT_DIR , 'keras_model'))

In [ ]:
model.save(os.path.join(OUTPUT_DIR , 'keras_model/my_model.h5'))